<a href="https://colab.research.google.com/github/HwangHanJae/Book-Recommendation/blob/main/Item_Base_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터로드

In [89]:
import pandas as pd
import numpy as np

rating_user_path = '/content/drive/MyDrive/Colab Notebooks/Recommendation/Book Recommendation/dataset/Rating_Users.csv'
rating_book_path = '/content/drive/MyDrive/Colab Notebooks/Recommendation/Book Recommendation/dataset/Rating_Books.csv'
new_ratings_path = '/content/drive/MyDrive/Colab Notebooks/Recommendation/Book Recommendation/dataset/New_Ratings.csv'

rating_users = pd.read_csv(rating_user_path)
rating_books = pd.read_csv(rating_book_path)
new_ratings = pd.read_csv(new_ratings_path)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Model

In [166]:
class Model():
  def __init__(self, rating, users, items, num_sim_item=20):
    self.R = rating
    self.users = set(users)
    self.items = set(items)
    self._get_mean()
    self.S = self._get_S()
    self.num_sim_item = num_sim_item
    

  def _get_mean(self):
    self.mean = {}
    for u in self.users:
      self.mean[u] = sum(self.R[u].values()) / len(self.R[u].keys())
    
  def _get_S(self):
    S = {}
    for u in self.R.keys():
      for i in self.R[u].keys():
        if u not in S:
          S[u] = {}
        S[u][i] = self.R[u][i] - self.mean[u]
    return S
  def _adjusted_cosine(self, i, j):
    if i == j:
      return 1

    # 아이템 i, j에 모두 평가한 유저의 인덱스
    numerator = 0
    denominator = 0
    sum_sui_2 = 0
    sum_suj_2 = 0
    for u in self.users:
      if self.S[u].get(i) and self.S[u].get(j):
    # 조정된 코사인 유사도 계산
        sui = self.S[u][i]
        suj = self.S[u][j]
        numerator += sui * suj

        sum_sui_2 += sui ** 2
        sum_suj_2 += suj ** 2

    sqrt_sum_sui = sum_sui_2 ** 0.5
    sart_sum_suj = sum_suj_2 ** 0.5

    denominator = sqrt_sum_sui * sart_sum_suj

    if denominator == 0:
      return 0
    else:
      return numerator / denominator

  def _get_peergroup(self, i):
    peergroup =[]
    for j in self.items - {i}:
      adjusted_cosine = self._adjusted_cosine(i, j)
      if adjusted_cosine > 0:
        peergroup.append((j, self._adjusted_cosine(i, j)))

    peergroup = sorted(peergroup, key=lambda x : x[1], reverse=True)[:self.num_sim_item]

    return peergroup
      
  def recommend(self, user_id, num_recom_items = 10):
    recommend_items = []
    for i in self.items - self.R[user_id].keys():
      peergroup = self._get_peergroup(i)
      numerator = 0
      denominator = 0
      for j, adjustedcosine in peergroup:
        numerator += (adjustedcosine * self.R[user_id][j])
        denominator += abs(adjustedcosine)
      if denominator == 0:
        ruj = self.mean[u]
      else:
        ruj = numerator / denominator
      recommend_items.append((i,ruj))
    recommend_items.sort(key=lambda x: x[1], reverse=True)

    return [item for item, rui in recommend_items[:num_recom_items]]


# Sample 데이터

In [167]:
array = [[7,6,7,4,5,4],
         [6,7,None,4,3,4],
         [None,3,3,1,1,None],
         [1,2,2,3,3,4],
         [1,None,1,2,3,3]]
sample_df = pd.DataFrame(array, index=[1,2,3,4,5], columns=[1,2,3,4,5,6])
sample_df

,1,2,3,4,5,6
1,7.0,6.0,7.0,4,5,4.0
2,6.0,7.0,NaN,4,3,4.0
3,NaN,3.0,3.0,1,1,NaN
4,1.0,2.0,2.0,3,3,4.0
5,1.0,NaN,1.0,2,3,3.0


In [168]:
sample_dict = {}
for u in sample_df.index:
  for i in sample_df.columns:
    if u not in sample_dict:
      sample_dict[u] = {}
    if ~np.isnan(sample_df.loc[u, i]):
      sample_dict[u][i] = sample_df.loc[u, i]


sample_dict

{1: {1: 7.0, 2: 6.0, 3: 7.0, 4: 4, 5: 5, 6: 4.0},
 2: {1: 6.0, 2: 7.0, 4: 4, 5: 3, 6: 4.0},
 3: {2: 3.0, 3: 3.0, 4: 1, 5: 1},
 4: {1: 1.0, 2: 2.0, 3: 2.0, 4: 3, 5: 3, 6: 4.0},
 5: {1: 1.0, 3: 1.0, 4: 2, 5: 3, 6: 3.0}}

In [169]:
users = set(sample_df.index)
items = set(sample_df.columns)
model = Model(sample_dict, users, items)

model.recommend(3)

[1, 6]

# 추천하기

In [170]:
rating_matrix = {}
for i in new_ratings.index:
  user = new_ratings.loc[i, 'User-ID']
  isbn = new_ratings.loc[i, 'ISBN']
  rating = new_ratings.loc[i, 'Book-Rating']
  if user not in rating_matrix:
    rating_matrix[user] = {}
  rating_matrix[user][isbn] = rating

In [171]:
users = new_ratings['User-ID'].unique()
items = new_ratings['ISBN'].unique()
model = Model(rating_matrix, users, items)

In [173]:
model.recommend(276721)

추천을 할 때 모든 아이템에 대하여 조정된 코사인 유사도를 구해야하기 때문에 아이템이 많아질 수록 시간이 오래걸리게 됩니다.  
따라서 미리 조정된 코사인 유사도를 구해놓는다면 추천하는데 시간이 줄어들 것 같습니다.